In [1]:
import pandas as pd
from collections import Counter

kospi = pd.read_excel("kospi100.xlsx", engine='openpyxl')

searches = kospi['종목명'][:5].tolist()
searches

C:\Users\ahn\anaconda3\envs\CapstonStockProject\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성바이오로직스', 'POSCO홀딩스']

In [2]:
senti_avg_list = []
keywords_list = []
senti_count_list = []
senti_count = 0

In [3]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import pandas as pd

from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&start=" + str(
            start_page)
        
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            #url = "https://search.naver.com/search.naver?where=news&query=%EC%B9%B4%EC%B9%B4%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.09.10&de=2021.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210910to20210915&is_sug_officeid=0"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url

# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'
# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
hannanum = Hannanum()

  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#####뉴스크롤링 시작#####

# 검색어 입력
# search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = 1
# 검색 종료할 페이지 입력
page2 = 10

for search in searches:
    # KoELECTRA 모델 로드
    model_name = "hyunwoongko/kobart"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 형태소 분석 함수
    hannanum = Hannanum()
    # naver url 생성
    url = makeUrl(search, page, page2)

    # 뉴스 크롤러 실행
    news_titles = []
    news_url = []
    news_contents = []
    news_dates = []

    for i in url:
        url = articles_crawler(url)
        news_url.append(url)



    # 제목, 링크, 내용 담을 리스트 생성
    news_url_1 = []

    # 1차원 리스트로 만들기(내용 제외)
    makeList(news_url_1, news_url)

    # NAVER 뉴스만 남기기
    final_urls = []
    for i in tqdm(range(len(news_url_1))):
        if "news.naver.com" in news_url_1[i]:
            final_urls.append(news_url_1[i])
        else:
            pass

    # 뉴스 내용 크롤링
    for i in tqdm(final_urls):
        # 각 기사 html get하기
        news = requests.get(i, headers=headers)
        news_html = BeautifulSoup(news.text, "html.parser")

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
    # ------------------------------------------------------------------------------------------------------------------
        # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one(
                "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
            news_date = news_date[0:10]
        # 날짜 가져오기
        news_dates.append(news_date)
    df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'content' : news_contents}) #'url' : final_urls
    
    # title 열에 대해 형태소 분석 적용

    df['title'] = df['title'].astype(str).apply(tokenize)
    # for i in range(df.shape[0]):
    #     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


    # title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
    df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))


    # train할 label은 제목을 읽고 내가 직접 라벨링
    # test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
    df['label'] = df['sentiment'].apply(convert_sentiment)
    df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
    df=round(df,2)
    # 데이터 프레임 저장
    now = datetime.datetime.now()
    date = str(now.year%100)+format(now.month,'02')+format(now.day,'02')
    #df.to_csv(f'csv/{date}_{search}_감성분석.csv', encoding='utf-8-sig', index=False)
#     평균으로 접근하면 너무 편차가 크다
    senti_avg = df['sentiment'].mean()
    senti_avg_list.append(senti_avg)
    
    
    if df[df['label'] == 1].label.count() > df[df['label'] == 0].label.count():
        senti_count = 1
    else:
        senti_count = 0
    senti_count_list.append(senti_count)


    
    sentimented_title = []
    for i in range(len(df['title'])):
        for j in range(len(df['title'][i])):
            if len(df['title'][i][j]) != 1: 
                sentimented_title.append(df['title'][i][j])

    counter = Counter(sentimented_title)

    keywords = []
    most_common_items = counter.most_common(5)  # 리스트의 길이를 먼저 확인
    for i in range(min(5, len(most_common_items))):
        keywords.append(most_common_items[i][0])
    keywords_list.append(keywords)

    
    


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:04<00:00,  6.72it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClas

In [6]:
print(keywords_list)
print(senti_avg_list)
print(senti_count_list)

[['삼성전자', '코스피', 'IFA', '용량', '2023'], ['코스피', 'LG', 'IFA', '시황', '마감'], ['코스피', '삼성전자', '마감', '하락', '상승'], ['코스피', '마감', '시황', '하락', '삼성전자'], ['코스피', '마감', '시황', '삼성전자', '2550대']]
[0.3549999999999999, 0.42200000000000004, 0.5970731707317073, 0.36892857142857144, 0.5994117647058823, 0.5647368421052631]
[0, 1, 0, 1, 1]


In [8]:
keys = searches
keywords_dict = dict(zip(keys, keywords_list))
senti_avg_dict = dict(zip(keys, senti_avg_list))
senti_count_dict = dict(zip(keys, senti_count_list))

{'삼성전자': 0.3549999999999999,
 'LG에너지솔루션': 0.42200000000000004,
 'SK하이닉스': 0.5970731707317073,
 '삼성바이오로직스': 0.36892857142857144,
 'POSCO홀딩스': 0.5994117647058823}

In [11]:
import requests

data = {
    'keywords': keywords_dict,
    'senti_avg': senti_avg_dict,
    'senti_count' : senti_count_dict 
}
#https://616d-39-118-146-59.ngrok-free.app/sentiment
server_url = 'http://127.0.0.1:5000/sentiment'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)

성공
